In [8]:
import os
import torch
from torch import nn

import numpy as np
from PIL import Image
import glob
import matplotlib as plt 

In [7]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [9]:
img = Image.open("mnist-subset/0/16585.png")
plt.imshow(img, cmap="gray") # display image as a greyscale

AttributeError: module 'matplotlib' has no attribute 'imshow'

In [10]:
num_classess = 10

def img_to_vec(img):
    """Return a vector representation of an MNIST image file"""
    img = Image.open(img)
    return np.array(img)

# D will house our data
D = []

# Iterate over all files that match the pattern "mnist-subset/*/*.png"
# and add its information to `D`. We will sort the filenames so that we get
# a consistent set of files in the training, validation, and test sets.
for file in sorted(glob.glob("mnist-subset/*/*.png")):
    x = img_to_vec(file)   # vector input
    t = file.split(os.sep)[1] # find out the target label by reading the file path
    D.append((x, t),) # add this to the data set

In [42]:
import random
random.shuffle(D)
D_train = D[:4000]     # the training set
D_valid = D[4000:4500] # the validation set
D_test  = D[4500:]     # the test set

In [43]:
X_train = np.stack([x for (x, _) in D_train])
t_train = np.eye(num_classess)[np.array([int(t) for (_, t) in D_train])]
X_valid = np.stack([x for (x, _) in D_valid])
t_valid = np.eye(num_classess)[np.array([int(t) for (_, t) in D_valid])]
X_test = np.stack([x for (x, _) in D_test])
t_test = np.eye(num_classess)[np.array([int(t) for (_, t) in D_test])]

In [52]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        self.flatten = nn.Flatten()
        self.linear_layers = nn.Sequential(
            nn.Linear(1568, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.flatten(x)
        x = self.linear_layers(x)
        return x
    
model = NeuralNetwork().to(device)

In [57]:
from torch import float32

learning_rate = 0.0008

data = torch.tensor(X_train, dtype=float32).cuda()
labels = torch.tensor(t_train, dtype=float32).cuda()

# Define loss function and optimizer
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
epochs = 50
batch_size = 64
for epoch in range(epochs):
    for i in range(0, len(data), batch_size):
        batch_data = data[i:i+batch_size, None]
        batch_labels = labels[i:i+batch_size]
        
        # Forward pass
        outputs = model(batch_data)

        # Compute loss
        loss = loss_fn(outputs, batch_labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print progress
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

print("Training finished!")

Epoch 1/50, Loss: 0.003377622691914439
Epoch 2/50, Loss: 0.00018797008669935167
Epoch 3/50, Loss: 0.015288724564015865
Epoch 4/50, Loss: 0.00036186864599585533
Epoch 5/50, Loss: 0.00043411817750893533
Epoch 6/50, Loss: 0.0004009395488537848
Epoch 7/50, Loss: 0.00010821464093169197
Epoch 8/50, Loss: 2.433669760648627e-05
Epoch 9/50, Loss: 3.043519200218725e-06
Epoch 10/50, Loss: 2.5219901544915047e-06
Epoch 11/50, Loss: 2.2761250875191763e-06
Epoch 12/50, Loss: 2.019084604398813e-06
Epoch 13/50, Loss: 1.750869159877766e-06
Epoch 14/50, Loss: 1.5534319572907407e-06
Epoch 15/50, Loss: 1.408147682013805e-06
Epoch 16/50, Loss: 1.2628634067368694e-06
Epoch 17/50, Loss: 1.110128891923523e-06
Epoch 18/50, Loss: 1.0207227205683012e-06
Epoch 19/50, Loss: 8.866141456564947e-07
Epoch 20/50, Loss: 7.934828545330674e-07
Epoch 21/50, Loss: 7.189776738414366e-07
Epoch 22/50, Loss: 6.519229600598919e-07
Epoch 23/50, Loss: 5.736926027566369e-07
Epoch 24/50, Loss: 5.252639994068886e-07
Epoch 25/50, Loss:

In [58]:
model.eval()

valid = torch.tensor(X_valid[:, None, :, :], dtype=torch.float32).cuda()

# Disable gradient calculation
with torch.no_grad():
    # Forward pass on validation data
    outputs = model(valid)
    
    # Get predicted labels
    _, predicted_val = torch.max(outputs, 1)
    
    results_val = predicted_val.detach().cpu().numpy()

    # Convert one-hot encoded labels back to integers
    t_valid_int = np.argmax(t_valid, axis=1)
    
    # Compute accuracy
    correct_val = np.sum(results_val == t_valid_int)
    accuracy_val = correct_val / len(t_valid_int)

print(f"Validation Accuracy: {accuracy_val * 100:.2f}%")


Validation Accuracy: 97.40%


In [59]:
model.eval()

test = torch.tensor(X_test[:, None, :, :], dtype=torch.float32).cuda()

# Disable gradient calculation
with torch.no_grad():
    # Forward pass on testing data
    outputs = model(test)
    
    # Get predicted labels
    _, predicted_test = torch.max(outputs, 1)
    
    results_test = predicted_test.detach().cpu().numpy()

    # Convert one-hot encoded labels back to integers
    t_test_int = np.argmax(t_test, axis=1)
    
    # Compute accuracy
    correct_test = np.sum(results_test == t_test_int)
    accuracy_test = correct_test / len(t_test_int)

print(f"Testing Accuracy: {accuracy_test * 100:.2f}%")


Testing Accuracy: 97.00%
